In [5]:
import mysql.connector
from mysql.connector.constants import ClientFlag

In [6]:
config = {
    'user': 'root',
    'password': 'Hh12H8y2ppBAq5Cj',
    'host': '34.65.175.109',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': '/Users/xavier/Desktop/EPFL_Cours/Ici/DB/db-54/part2/ssl/server-ca.pem',
    'ssl_cert': '/Users/xavier/Desktop/EPFL_Cours/Ici/DB/db-54/part2/ssl/client-cert.pem',
    'ssl_key': '/Users/xavier/Desktop/EPFL_Cours/Ici/DB/db-54/part2/ssl/client-key.pem',
    'database': 'data'
}

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

# Party
## Main Tables

In [49]:
cursor.execute("""CREATE TABLE Parties(id INTEGER,
                        case_id INTEGER NOT NULL,
                        party_number INTEGER NOT NULL,
                        finanicial_responsibility INTEGER,
                        party_age INTEGER,
                        party_sex INTEGER,
                        at_fault BIT NOT NULL,
                        cellphone_use INTEGER,
                        hazardous_materials CHAR(1),
                        movement_preceding_collision INTEGER,
                        other_associate_factor_1 INTEGER,
                        other_associate_factor_2 INTEGER,
                        party_drug_physical INTEGER,
                        party_safety_equipment_1 INTEGER,
                        party_safety_equipment_2 INTEGER,
                        party_sobriety INTEGER,
                        PRIMARY KEY(id),
                        FOREIGN KEY(finanicial_responsibility) REFERENCES FinancialResponsability(id),
                        FOREIGN KEY(party_sex) REFERENCES PersonSex(id),
                        FOREIGN KEY(cellphone_use) REFERENCES CellphoneUse(id),
                        FOREIGN KEY(movement_preceding_collision) REFERENCES MovementPrecedingCollision(id),
                        FOREIGN KEY(other_associate_factor_1) REFERENCES OtherAssociatedFactors(id),
                        FOREIGN KEY(other_associate_factor_2) REFERENCES OtherAssociatedFactors(id),
                        FOREIGN KEY(party_drug_physical) REFERENCES PartyDrugPhysical(id),
                        FOREIGN KEY(party_safety_equipment_1) REFERENCES PartySafetyEquipement(id),
                        FOREIGN KEY(party_safety_equipment_2) REFERENCES PartySafetyEquipement(id),
                        FOREIGN KEY(party_sobriety) REFERENCES PartySobriety(id),
                        FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")

In [50]:
cursor.execute("""CREATE TABLE Vehicles(party_id INTEGER NOT NULL,
                        school_bus_related BIT,
                        statewide_vehicle_type INTEGER,
                        vehicle_make INTEGER,
                        vehicle_year INTEGER,
                        FOREIGN KEY(statewide_vehicle_type) REFERENCES StatewideVehiculeType(id),
                        FOREIGN KEY(vehicle_make) REFERENCES VehiculeMake(id),
                        FOREIGN KEY(party_id) REFERENCES Parties(id))""")

## Satelite Enum Tables

In [11]:
cursor.execute("DROP TABLE OtherAssociatedFactors")
cursor.execute("CREATE TABLE OtherAssociatedFactors (id INTEGER,"
    "description CHAR(1) NOT NULL UNIQUE,"
    "CHECK (description BETWEEN 'A' AND 'Z'),"
    "PRIMARY KEY(id))")

In [12]:
cursor.execute("DROP TABLE CellphoneUse")
cursor.execute("""CREATE TABLE CellphoneUse(id INTEGER,
			   description CHAR(1) NOT NULL UNIQUE,
                CHECK (description BETWEEN 'B' AND 'D'),	
			   PRIMARY KEY(id))""")
# messy data: 1 -> B, 2 -> C, 3 -> D

In [14]:
cursor.execute("""CREATE TABLE FinancialResponsability(id INTEGER,
			   	      description CHAR(1) NOT NULL UNIQUE,
                                      CHECK (description IN ('N', 'Y', 'O', 'E', '-')),	
			              PRIMARY KEY(id))""")

# clean data 

In [16]:
cursor.execute("""CREATE TABLE MovementPrecedingCollision(id INTEGER,
					 description VARCHAR(100) NOT NULL UNIQUE,
					 PRIMARY KEY(id))""")
# varchar -> messy data, whatever.

In [17]:
cursor.execute("""CREATE TABLE PartyDrugPhysical(id INTEGER,
			       description CHAR(1) NOT NULL UNIQUE,
                            CHECK (description IN ('E', 'F', 'H', 'I')),
			       PRIMARY KEY(id))""")

# messy: letter G most prevalent: interpret as n/a ?

In [18]:
cursor.execute("""CREATE TABLE PartySafetyEquipement(id INTEGER,
				   description CHAR(1) NOT NULL UNIQUE,
                                   CHECK (description BETWEEN 'A' AND 'Y'),
				   PRIMARY KEY(id))""")
# clean as far as i can tell

The following table holds for `VictimSex` and `PartySex`

In [37]:
cursor.execute("""CREATE TABLE PersonSex(id INTEGER,
		       description VARCHAR(6) NOT NULL UNIQUE,
                       CHECK (description in ('male', 'female')),
		       PRIMARY KEY(id))""")
# clean af, for PartySex and VictimSex

In [20]:
cursor.execute("""CREATE TABLE PartySobriety(id INTEGER,
			    description CHAR(1) NOT NULL UNIQUE,
                            CHECK (description IN ('A', 'B', 'C', 'D', 'G', 'H')),
			    PRIMARY KEY(id))""")
# clean af

In [22]:
cursor.execute("""CREATE TABLE PartyType(id INTEGER,
			description VARCHAR(15) NOT NULL UNIQUE,
			PRIMARY KEY(id))""")

# clean

In [23]:
cursor.execute("""CREATE TABLE StatewideVehiculeType(id INTEGER,
				    description VARCHAR(30) NOT NULL UNIQUE,
				    PRIMARY KEY(id))""")
# clean as far as i can tell

In [24]:
cursor.execute("""CREATE TABLE VehiculeMake(id INTEGER,
			   description VARCHAR(100) NOT NULL,
			   PRIMARY KEY(id))""")

# clean

# Collision
## Main Tables

In [42]:
cursor.execute("""CREATE TABLE Collisions(case_id INTEGER, 
                        collision_date DATE NOT NULL,
                        collision_time TIME,
                        type_of_collision INTEGER,
                        collision_severity INTEGER NOT NULL,
                        hit_and_run INTEGER NOT NULL,
                        tow_away BIT,
                        FOREIGN KEY(type_of_collision) REFERENCES TypeOfCollision(id), 
                        FOREIGN KEY(collision_severity) REFERENCES CollisionSeverity(id), 
                        FOREIGN KEY(hit_and_run) REFERENCES HitAndRun(id),                      
                        PRIMARY KEY(case_id))""")

In [43]:
cursor.execute("""CREATE TABLE Pcfs(case_id INTEGER NOT NULL,
                    pcf_violation INTEGER,
                    pcf_violation_category INTEGER,
                    pcf_violation_subsection CHAR(1),
                    FOREIGN KEY(pcf_violation_category) REFERENCES PcfViolationCategory(id),
                    FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")
# pcf_violation_subscection has weird values but we cannot infer
# that they are wrong.

In [44]:
cursor.execute("""CREATE TABLE Locations(case_id INTEGER NOT NULL,
                        population INTEGER,
                        county_city_location INTEGER NOT NULL,
                        FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")

In [45]:
cursor.execute("""CREATE TABLE Factors(case_id INTEGER NOT NULL,
                        location_type INTEGER,
                        lighting INTEGER,
                        road_condition_1 INTEGER,
                        road_condition_2 INTEGER,
                        road_surface INTEGER,
                        weather_1 INTEGER,
                        weather_2 INTEGER,
                        FOREIGN KEY(location_type) REFERENCES LocationType(id),
                        FOREIGN KEY(lighting) REFERENCES Lighting(id),
                        FOREIGN KEY(road_condition_1) REFERENCES RoadCondition(id),
                        FOREIGN KEY(road_condition_2) REFERENCES RoadCondition(id),
                        FOREIGN KEY(road_surface) REFERENCES RoadSurface(id),
                        FOREIGN KEY(weather_1) REFERENCES Weather(id),
                        FOREIGN KEY(weather_2) REFERENCES Weather(id),
                        FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")

In [46]:
cursor.execute("""CREATE TABLE Cases(case_id INTEGER NOT NULL,
                    process_date DATE NOT NULL,
                    officer_id VARCHAR(8),
                    jurisdiction INTEGER,
                    FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")
# max officer_id length is 8. As some include letters, better make it VARCHAR.

## Satelite Enum Tables

In [25]:
cursor.execute("""CREATE TABLE CollisionSeverity(id INTEGER,
			       description VARCHAR(25) NOT NULL UNIQUE,
			       PRIMARY KEY(id))""")
# clean

In [26]:
cursor.execute("""CREATE TABLE HitAndRun(id INTEGER,
			description VARCHAR(20) NOT NULL UNIQUE,
			PRIMARY KEY(id))""")
# One dirty value: D -> NaN 

In [27]:
cursor.execute("""CREATE TABLE Lighting(id INTEGER,
		       description VARCHAR(35) NOT NULL UNIQUE,
		       PRIMARY KEY(id))""")
# Clean

In [28]:
cursor.execute("""CREATE TABLE LocationType(id INTEGER,
			   description VARCHAR(20) NOT NULL UNIQUE,
			   PRIMARY KEY(id))""")
# clean

In [29]:
cursor.execute("""CREATE TABLE PcfViolationCategory(id INTEGER,
				   description VARCHAR(70) NOT NULL UNIQUE,
				   PRIMARY KEY(id))""")

# One dirty value: 21804 -> NaN

In [30]:
cursor.execute("""CREATE TABLE PrimaryCollisionFactor(id INTEGER,
				     description VARCHAR(25) NOT NULL UNIQUE,
				     PRIMARY KEY(id))""")
# clean

In [31]:
cursor.execute("""CREATE TABLE RoadCondition(id INTEGER,
			    description VARCHAR(20) NOT NULL UNIQUE,
			    PRIMARY KEY(id))""")
# clean

In [32]:
cursor.execute("""CREATE TABLE RoadSurface(id INTEGER,
			  description VARCHAR(15) NOT NULL UNIQUE,
			  PRIMARY KEY(id))""")
# one dirty value: H -> NaN

In [33]:
cursor.execute("""CREATE TABLE TypeOfCollision(id INTEGER,
			      description VARCHAR(15) NOT NULL UNIQUE,
			      PRIMARY KEY(id))""")
# clean

In [34]:
cursor.execute("""CREATE TABLE Weather(id INTEGER,
		      description VARCHAR(10) NOT NULL UNIQUE,
		      PRIMARY KEY(id))""")
# clean

# Victim
## Main Table

In [53]:
cursor.execute("""CREATE TABLE Victims(id INTEGER,
                        case_id INTEGER NOT NULL,
                        party_number INTEGER NOT NULL,
                        victim_age INTEGER,
                        victim_sex INTEGER,
                        victim_degree_of_injury INTEGER,
                        victim_ejected INTEGER,
                        victim_role INTEGER NOT NULL,
                        victim_safety_equipment_1 INTEGER,
                        victim_safety_equipment_2 INTEGER,
                        victim_seating_position INTEGER,
                        PRIMARY KEY(id),
                        FOREIGN KEY(victim_sex) REFERENCES PersonSex(id),
                        FOREIGN KEY(victim_safety_equipment_1) REFERENCES VictimSafetyEquipment(id),
                        FOREIGN KEY(victim_safety_equipment_2) REFERENCES VictimSafetyEquipment(id),
                        FOREIGN KEY(victim_degree_of_injury) REFERENCES VictimDegreeOfInjury(id),
                        FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")

## Satelite Table

In [52]:
cursor.execute("""CREATE TABLE VictimSafetyEquipment(id INTEGER,
				   description CHAR(1) NOT NULL UNIQUE,
                                   CHECK(description BETWEEN 'A' AND 'Z'),
				   PRIMARY KEY(id))""")

In [36]:
cursor.execute("""CREATE TABLE VictimDegreeOfInjury(id INTEGER,
				   description VARCHAR(30) NOT NULL UNIQUE,
				   PRIMARY KEY(id))""")
# one drity value: 7 -> NaN